# 랜덤포레스트

 ## 배깅(Bagging)
 
 - bootstrap aggregation의 약자
 - 표본에서 추가적 표본 복원 추출해서 각 표본에 대한 통계량 계산하는 과정
 
 ## 부스팅(Boosting)
 
 - 가중치를 이용하여 강 / 약 분류기로 만드는 방법
 - 각각의 독립적인 모델들을 결합하여 최종 결과 값을 예측하는 것
 

# XGBoost

## 장점
 ### 1) 병렬처리를 통한 빠른 학습
 ### 2) 유연한 learning system
 ### 3) overfitting 방지 위한 설계
 ### 4) 다양한 시나리오에 대한 확장성

<img src="img_01.png" width=600, height=600>

# 파라미터
 - sub_sample[default = 1] = 트리가 커져서 과적합 되는 것을 제어하기 위해 데이터 샘플링하는 비율 지정. 0.5 ~ 1사이 값을 사용.
 - booster = 'gbtree' #트리, 회귀(gblinear) 트리가 항상 더 좋은 성능을 내기 때문에 수정 필요 없음.
 - max_depth # 트리의 최대 깊이 정의.
 - gamma = 0 # 트리의 리프 노드를 추가적으로 나눌지 결정할 최소 손실 감소 값. 해당 값보다 큰 손실(loss)이 감소된 경우
           리프노드를 분리. 값이 클수록 과적합 감소 효과가 있다.
 - nthred = 4 # XGBoost를 실행하기위한 병렬처리(쓰레드) 갯수, 'n_jobs'사용.
 - n_estimator = int # 부스트 트리의 개수.
 - objective = 'reg:linear' 회귀/ 'binary:logstic' 분류 / 'multi:softmax' 다중분류/ 'multi:softprob' 각 class에 속할 확률 return
 - random_state = seed와 동일
 - silent = 0(default) -> 출력메시지 나타내고 싶지 않을 경우 1로 설정.
 - eta = 0.3(default) -> 학습률 파라미터. 0~1 값 지정. (0.01 ~ 0.2 사이 값 선호)
 
 ### fit - early_stopping_round = 100, 20 / 100번, 20번 반복동안 성능이 좋아지지 않으면 stop
 ### fit - eval_metric = 검증에 사용되는 함수 정의 

감마 [default = 0] [범위 : (0, Inf)]
정규화를 제어합니다 (또는 과적 합 방지). 감마의 최적 값은 데이터 세트 및 기타 매개 변수 값에 따라 다릅니다.
값이 높을수록 정규화가 높아집니다. 정규화는 모델의 성능을 향상시키지 않는 큰 계수에 대한 처벌을 의미합니다. default = 0정규화가 없음을 의미합니다.
튜닝 트릭 : 0으로 시작하고 CV 오류율을 확인하십시오. 열차 오류 >>> 테스트 오류가 표시되면 감마를 작동시킵니다. 감마가 높을수록 열차와 테스트 CV의 차이가 줄어 듭니다. 사용할 값에 대한 단서가 없으면 gamma = 5를 사용하고 성능을 확인하십시오. 얕은 (낮은 max_depth) 나무 를 사용하려는 경우 감마가 향상됩니다 .

In [4]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd
# 랜덤포레스트 분류기 파라미터 최적화
from sklearn.model_selection import GridSearchCV

In [2]:
# 유방암? 데이터 적재
cancer = load_breast_cancer()

# 유방암 데이터 DataFrame으로 만든 뒤, target 칼럼 생성
cancer_df = pd.DataFrame(data = cancer.data, columns = cancer.feature_names)
cancer_df['target'] = cancer.target

# 학습/검증 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(cancer_df.iloc[:, :-1], cancer_df.target)
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
import xgboost
from xgboost import XGBClassifier

# XGBoost 분류기 생성
xgb_clf = XGBClassifier()

# XGBoost 분류기 학습
xgb_clf.fit(X_train, y_train)

# XGBoost 분류기 평가
xgb_clf.score(X_test, y_test)

0.958041958041958

In [ ]:
xgb_clf = XGBClassifier(booster = 'gbtree',
                        subsample = '0.5',
                        eta = '0.2',
                        gamma = 0, 
                        max_depth= 8,
                        nthread = 4,
                        objective = 'binary:logistic',
                        random_state = 2,
                        silent = True)

In [13]:
# XGBoost 분류기 파라미터 최적화

# 파라미터 시험값들 사전으로 정의
xgb_params = {
    'gamma' : [0, 1, 3, 5, 7],
    'max_depth' : [3, 5, 7, 9],
    'n_estimators' :[50, 100, 200],
    'learning_rate' : [0.01, 0.05, 0.1, 0.2]
}

# 파라미터 최적화
grid_xgb = GridSearchCV(xgb_clf, param_grid = xgb_params, cv = 5, n_jobs = -1)

# 파라미터를 최적화 한 XGBoost 분류기 다시 훈련
grid_xgb.fit(X_train, y_train)

# 파라미터를 최적화 한 XGBoost 분류기 평가
grid_xgb.fit(X_test, y_test)    # 0.9370629370629371

# 최적 파라미터 찾기
print(grid_xgb.best_params_) # {'learning_rate': 0.2, 'n_estimators': 100}

{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


C:\Users\82106\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


AttributeError: 'GridSearchCV' object has no attribute 'best_socres_'

In [17]:
print(grid_xgb.best_score_)
print(grid_xgb.best_estimator_)

0.972027972027972
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
